# ***YOLOv12 + R-ELAN + FPN***

In [1]:
!pip install -q ultralytics roboflow

import os
from pathlib import Path
import yaml
from roboflow import Roboflow
from ultralytics import YOLO

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 10

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# Custom YAML config untuk YOLO12n dengan FPN Neck (hanya top-down)
custom_yaml_content = """
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]  # YOLO12n
  s: [0.50, 0.50, 1024]  # YOLO12s
  m: [0.50, 1.00, 512]   # YOLO12m
  l: [1.00, 1.00, 512]   # YOLO12l
  x: [1.00, 1.50, 512]   # YOLO12x

# Backbone
backbone:
  - [-1, 1, Conv, [64, 3, 2]]    # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]   # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, Conv, [256, 3, 2]]   # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, Conv, [512, 3, 2]]   # 5-P4/16
  - [-1, 4, A2C2f, [512, True, 4]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 4, A2C2f, [1024, True, 1]]  # 8

# Head dengan FPN Neck (top-down only)
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 9
  - [[-1, 6], 1, Concat, [1]]                   # 10
  - [-1, 2, A2C2f, [512, False, -1]]            # 11
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 12
  - [[-1, 4], 1, Concat, [1]]                   # 13
  - [-1, 2, A2C2f, [256, False, -1]]            # 14
  - [[14, 11, 8], 1, Detect, [nc]]              # 15
"""

custom_yaml_path = Path("/content/yolo12n_r-elan_fpn.yaml")
with open(custom_yaml_path, "w") as f:
    yaml.dump(yaml.safe_load(custom_yaml_content), f)
print(f"✅ Custom YAML FPN dibuat: {custom_yaml_path}")

model = YOLO(str(custom_yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_r-elan_fpn", epochs=50):
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_fpn/weights/best.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dental-caries-10 in yolov12:: 100%|██████████| 16480/16480 [00:03<00:00, 5405.26it/s]


✅ Dataset berhasil diunduh: /content/dental-caries-10/data.yaml
Struktur dataset: ['train', 'valid', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']
✅ Custom YAML FPN dibuat: /content/yolo12n_fpn.yaml
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dental-caries-10/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.

KeyboardInterrupt: 